### Imports

In [ ]:
import datetime
import os

import geopandas as gpd
import pandas as pd
import partridge as ptg

### Read in the GTFS Files
Specify the path to the GTFS folder.

In this case, the script is selecting the busiest day of the GTFS to use as it's guide.

In [ ]:
gtfs_feed = 'carta_2019_05_02'
gtfs_path = os.path.join('..', 'data', 'gtfs', gtfs_feed)

service_date, service_ids = ptg.read_busiest_date(gtfs_path)

print('Selected Date: {}'.format(service_date))

view = {
    'trips.txt': {'service_id': service_ids},
}

feed = ptg.load_geo_feed(gtfs_path, view)

### Merge (Spatial Attribute Join) GTFS Shapes and Trips

In [ ]:
trip_fields = ['route_id', 'trip_headsign', 'direction_id', 'shape_id']
route_shapes = feed.shapes.merge(feed.trips.drop_duplicates(trip_fields)[trip_fields],
                                 on='shape_id', how='left')

### Specify the DBF Schema and Export to Shapefile

In [ ]:
schema = {'geometry': 'LineString',
          'properties':
              {'shape_id': 'str:15', 
               'route_id': 'str:25',
               'trip_headsign': 'str:25',
               'direction_id': 'int'
              }
         }

route_shapes.to_file(os.path.join('..', 'data', 'gtfs', 'route_shapes_{}.shp'.format(gtfs_feed)),
                     schema=schema)